In [2]:
%matplotlib widget
%load_ext line_profiler
import raster_geometry as rg
import numpy as np
from scipy import spatial, stats
from functools import reduce
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp


The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [3]:
# # size = (100, 100, 100)
# radius = 1
# size = 2 * (radius + 1)
# # position = (50, 50, 50)

# midpoint = [size / 2] * 3
# print(midpoint)

# # target = rg.nd_superellipsoid(size, radius, position=position, rel_sizes=False, rel_position=False)
# print(f"Radius: {radius}, Container size: {size}")
# sphere = rg.nd_superellipsoid(size, radius, position=midpoint, rel_sizes=False, rel_position=False).astype(np.int_)
# print(sphere)
# # sphere = rg.sphere(size, radius, 0.5).astype(np.int_)

In [4]:

# print("GPU")
# %time test_superellipsoid((5,5,5), 2, rel_sizes=False).astype(np.int_)
# %time test_superellipsoid((50,50,50), 2, rel_sizes=False).astype(np.int_)
# %time test_superellipsoid((500,500,500), 2, rel_sizes=False).astype(np.int_)
# %time test_superellipsoid((1000,1000,1000), 2, rel_sizes=False).astype(np.int_)
# print()
# print("CPU")
# %time rg.nd_superellipsoid((5,5,5), 2, rel_sizes=False).astype(np.int_)
# %time rg.nd_superellipsoid((50,50,50), 2, rel_sizes=False).astype(np.int_)
# %time rg.nd_superellipsoid((500,500,500), 2, rel_sizes=False).astype(np.int_)
# %time rg.nd_superellipsoid((1000,1000,1000), 2, rel_sizes=False).astype(np.int_)
# # 

In [5]:
def draw_sphere_in_volume(volume: np.ndarray, radius: int, position: tuple) -> None:
    """
    Draw a sphere in a given 3D NumPy array at a specified position.

    Args:
    volume (numpy.ndarray): The 3D NumPy array in which the sphere will be drawn.
    radius (int): The radius of the sphere.
    position (tuple): A 3-tuple containing the z, y, x coordinates of the position in the array where the sphere will be drawn.

    Returns:
    None
    """
    # Create an empty 3D NumPy array with dimensions equal to twice the radius plus one
    size = 2 * (radius + 1)

    # Calculate the midpoint of the sphere unit array
    midpoint = [size / 2] * 3

    # Generate a unit sphere using the rg library's superellipsoid function
    sphere = rg.nd_superellipsoid(size, radius, position=midpoint,
                                  rel_sizes=False, rel_position=False).astype(np.int_)

    # Extract the z, y, x coordinates of the position where the sphere will be drawn
    z, y, x = map(round, position)

    # Calculate the delta change needed to center the sphere at the specified position
    d = (size//2)

    # Calculate the minimum and maximum indices for the z, y, x axes of the volume array
    # print(position, radius, x-d, x+d, max(x - d, 0), min(x + d, volume.shape[2]))
    zmin, zmax = max(z - d, 0), min(z + d, volume.shape[0])
    ymin, ymax = max(y - d, 0), min(y + d, volume.shape[1])
    xmin, xmax = max(x - d, 0), min(x + d, volume.shape[2])

    # Calculate the minimum indices for the z, y, x axes of the sphere array
    szmin = abs(z - d) if z - d < 0 else 0
    symin = abs(y - d) if y - d < 0 else 0
    sxmin = abs(x - d) if x - d < 0 else 0

    # Calculate the amount to cut off of the ends of the z, y, x axes of the sphere array
    szmax = abs(volume.shape[0] - (z + d)) if z + d > volume.shape[0] else 0
    symax = abs(volume.shape[1] - (y + d)) if y + d > volume.s100hape[1] else 0
    sxmax = abs(volume.shape[2] - (x + d)) if x + d > volume.shape[2] else 0
    

    # Trim the sphere array to fit within the trimmed volume array
    try:
        sphere = sphere[szmin:sphere.shape[0]-szmax, symin:sphere.shape[1]-symax, sxmin:sphere.shape[2]-sxmax]
    except:
        print("Z", szmin, szmax, "Y", symin, symax, "X", sxmin, sxmax)

    # Place the sphere within the larger volume array at the specified position
    volume[zmin:zmax, ymin:ymax, xmin:xmax] = sphere

In [6]:
class Ripley():
    def __init__(
        self,
        points: np.ndarray,
        radii: list,
        volume_shape: tuple,
        boundary_correction: bool = True
    ):
        self.points = points
        self.radii = radii
        self.volume_shape = volume_shape
        self.boundary_correction = boundary_correction
        self._validate_inputs()

        self.results = {"K": [], "L": [], "H": []}

        self.tree = spatial.cKDTree(self.points)
        self.study_volume = reduce(lambda x, y: x * y, self.volume_shape)

    def run_ripley(self, processes=32):
        with mp.Pool(processes) as pool:
            pool.map(self._calc_ripley, self.radii)
            # list(tqdm(pool.imap(self._calc_ripley, self.radii), total=len(radii)))

        # print("done")
        # print(self.results["K"])

    def _validate_inputs(self):
        # Check if self.points is a list or numpy array
        if not isinstance(self.points, (list, np.ndarray)):
            e = f"Expected {np.ndarray}, received {type(self.points)}"
            raise ValueError(e)

        # Convert self.points to numpy array if it is a list
        if not isinstance(self.points, np.ndarray):
            self.points = np.array(self.points)

        # Check if self.points array has two dimensions
        if len(self.points.shape) != 2:
            e = f"Expected self.points array to have 2 dimensions, but got array with shape {self.points.shape}"
            raise ValueError(e)

        # Check if the self.points array second dimension length is 3 (x, y, z)
        if self.points.shape[1] != 3:
            e = f"Expected self.points array to have shape (None, 3), but got array with shape {self.points.shape}"
            raise ValueError(e)

        # Check if the self.points array has at least 3 points
        if self.points.shape[0] < 3:
            e = f"Expected self.points array to have at least 3 points"
            raise ValueError(e)

        # Check if radii is list-like or number-like
        if not isinstance(self.radii, (np.ndarray, list, int, float)):
            e = f"Expected {(np.ndarray, list, int, float)}, received {type(self.radii)}"
            raise ValueError(e)

        # if only one radius given as int, convert to list
        if not isinstance(self.radii, (np.ndarray, list)):
            self.radii = [self.radii]

    def _calc_ripley(self, radius):
        # For each radius, loop through each point and count points
        # within the radius
        nb_count = 0
        for z, y, x in self.points:
            if self.boundary_correction:
                weight = self.calculate_weight(radius, (z, y, x))
                # If weight is zero (i.e. target sphere not in mask), move on
                if weight == 0:
                    continue
            else:
                weight = 1.0

            # query_ball_point() includes the index of the current point as well
            # so 1 is subtracted from the count
            nb_count += (len(self.tree.query_ball_point([z, y, x], radius)) - 1) / weight
            pbar.update()


         # calculating 3D Ripley's functions (K, L, H)
        N = self.points.shape[0]
        K = nb_count * self.study_volume / (N * (N - 1))
        L = ((3. / 4) * (K / np.pi)) ** (1. / 3)
        H = L - radius
        
        # Verify K/L values positive
        if K < 0 or L < 0:
            raise ValueError(f"K/L values should not be negative. nb_count: {nb_count}, volume: {volume}, N: {N}")

        self.results["K"].append((radius, K))
        self.results["L"].append((radius, L))
        self.results["H"].append((radius, H))

    def calculate_weight(self, radius, center):
        """
        Calculate the proportion of a sphere within a study volume.

        Args:
        radius (int): The radius of the sphere.
        center (tuple): A 3-tuple containing the z, y, x coordinates of the center of the sphere.

        Returns:
        float: The proportion of the sphere within the study volume.
        """
        # Ensure that the radius is greater than zero
        assert radius > 0

        # Draw the target sphere in a 3D NumPy array at the specified position
        target = np.zeros(self.volume_shape)
        draw_sphere_in_volume(target, radius, center)

        # Calculate the sum (volume) of the target and reference sphere arrays
        target = target.sum()

        # Reference is calculated instead of simulated for speed increase
        reference = (4 / 3) * np.pi * (radius ** 3)

        # Ensure that the reference sphere has a non-zero volume
        assert reference > 0

        # Return the proportion of the sphere within the study volume
        # Since target volume is estimated whereas reference volume is calculated,
        # there is a small but noticable margin of error when radius < 10, there
        # we set maxiumum value of 1.0
        return min(target / reference, 1.0)


In [7]:
CSR_data = []
CSR_data_w = []
radii=np.arange(500, 510) 

np.random.seed(69420)
CSR_points = stats.uniform.rvs(0, 100, (100,3))

global pbar
pbar = tqdm(total=len(CSR_points))

ripley = Ripley(CSR_points, radii, (100,3000,3000), boundary_correction=True)
%time ripley.run_ripley(32)


 12%|████████▊                                                                | 12/100 [01:38<11:57,  8.15s/it]Process ForkPoolWorker-20:
Process ForkPoolWorker-21:
Process ForkPoolWorker-13:
Process ForkPoolWorker-11:
Process ForkPoolWorker-16:
Process ForkPoolWorker-28:
Process ForkPoolWorker-32:
Process ForkPoolWorker-14:
Process ForkPoolWorker-19:
Process ForkPoolWorker-29:
Process ForkPoolWorker-15:
Process ForkPoolWorker-26:
Process ForkPoolWorker-12:
Process ForkPoolWorker-22:
Process ForkPoolWorker-18:
Process ForkPoolWorker-27:
Process ForkPoolWorker-24:
Process ForkPoolWorker-23:
Process ForkPoolWorker-17:
Process ForkPoolWorker-25:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):

In [ ]:
def set_aspect_ratio_equal(ax) -> None:
    xlim = ax.get_xlim3d()
    ylim = ax.get_ylim3d()
    zlim = ax.get_zlim3d()
    ax.set_box_aspect((xlim[1]-xlim[0], ylim[1]-ylim[0], zlim[1]-zlim[0]))

z,x,y = volume.nonzero()
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
set_aspect_ratio_equal(ax)
ax.axes.set_xlim3d(left=0, right=volume.shape[0]) 
ax.axes.set_ylim3d(bottom=0, top=volume.shape[1]) 
ax.axes.set_zlim3d(bottom=0, top=volume.shape[2]) 
ax.scatter(x, y, z, zdir='z', c= 'red')
plt.show()